In [1]:
import numpy
from matplotlib import pyplot
from pandas import read_csv
import math
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
%matplotlib inline
dataframe1 = read_csv('newDailyData.csv', engine='python')
dataframe = dataframe1.iloc[88:,2:]
dataset = dataframe.values
# 将整型变为float
dataset = dataset.astype('float64')
dataframe.head()

,cmeclose,cmcclose,google,ravenpack,XAUprice,cmcvolume,market,AveBlockSize,AveTimeMaBlock,AveTran,...,SMA,RSI,TRIX,ROC,macd,macdsignal,macdhist,ADX,ATR,CCI
88,8525,7916.88,11,-0.26495,1314.00,4426149888,1.400000e+11,0.515428,9.757606,1036.885906,...,10075.114333,34.059231,-0.406550,-20.557680,-562.998249,-323.773162,-239.225087,27.230875,5249.784653,23.037639
89,8525,8223.68,13,-0.25410,1314.00,6639190016,1.320000e+11,0.695280,9.535099,1088.748344,...,10008.107000,37.659176,-0.415854,-12.467576,-584.029909,-375.824512,-208.205398,28.866723,5044.276463,67.378755
90,8425,8630.65,12,-0.22823,1316.94,6729110016,1.410000e+11,0.678045,9.210215,1235.664516,...,9925.372000,42.169334,-0.424889,-7.570508,-561.387251,-412.937059,-148.450191,29.957411,4880.468144,118.341588
91,8920,8913.47,12,-0.17697,1311.21,6361789952,1.440000e+11,0.793883,11.406955,1536.755906,...,9870.761000,45.139699,-0.432832,0.535416,-514.688561,-433.287360,-81.401201,29.302440,4780.378277,82.767388
92,8895,8929.28,11,-0.15176,1332.18,6043129856,1.510000e+11,0.670269,9.455373,1281.881579,...,9794.227000,45.308811,-0.439656,-6.779153,-470.974675,-440.824823,-30.149852,29.095382,4680.523400,121.752525


In [2]:
dataset.shape

(746, 40)

In [3]:
labels1 = dataframe1.iloc[:,1:2].values
scaler1 = OneHotEncoder()
labels = scaler1.fit_transform(labels1)

labels=labels.toarray()

In [4]:
def create_dataset(dataset,dataset1,dataset2,look_back):
    dataX, dataY = [], []
    for i in range(0,len(dataset)-look_back,1):
        a1 = dataset[i:(i+look_back),:]
        a2 = dataset1[i:(i+look_back),:]
        a = numpy.hstack((a1,a2))
        dataX.append(a)
        b = dataset1[(i+look_back):(i+look_back+1),1]
        dataY.append(b)
    return numpy.array(dataX), numpy.array(dataY)

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [6]:
look_back = 20
look_lag = 1 #用前20个数据预测当前1个数据
train_size = 600
test_size = len(dataset) - train_size
train, test = dataset[:train_size,:], dataset[train_size-look_back:len(dataset),:]
train1, test1 = labels[:train_size,:], labels[train_size-look_back:len(dataset),:]
train2, test2 = labels1[:train_size,:], labels1[train_size-look_back:len(dataset),:]
train1, test1 = labels[:train_size,:], labels[train_size-look_back:len(dataset),:]
trainX, trainY = create_dataset(train,train1,train2,look_back)
testX, testY = create_dataset(test, test1 ,test2,look_back)
print(trainX.shape)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], look_back,42))
testX = numpy.reshape(testX, (testX.shape[0],look_back,42))

features_set = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1]*42))
scaler1 = StandardScaler().fit(features_set) #MinMaxScaler(feature_range=(-1, 1))
#features_set = scaler1.fit_transform(features_set)
#pca=PCA(n_components=500).fit(features_set)
#features_set = pca.transform(features_set)
test_features = numpy.reshape(testX, (testX.shape[0], testX.shape[1]*42))
#test_features = scaler1.fit_transform(test_features)
#test_features = pca.transform(test_features)

def result(testY,testPredict):
    from sklearn.metrics import accuracy_score,classification_report
    acc = accuracy_score(testY,testPredict)
    report = classification_report(testY,testPredict)
    print("acc:",acc)
    print(report)

trainY = trainY.reshape(trainY.shape[0])
testY = testY.reshape(testY.shape[0])
trainY

(580, 20, 42)


array([0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 0., 1.

In [7]:
from sklearn.linear_model import LogisticRegression as LR
model1 = LR() #C=100,max_iter=1000
model1.fit(features_set, trainY)
testPredict1 = model1.predict(test_features)  

/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [8]:
result(testY,testPredict1)

acc: 0.5616438356164384
              precision    recall  f1-score   support

         0.0       0.59      0.81      0.68        84
         1.0       0.47      0.23      0.30        62

    accuracy                           0.56       146
   macro avg       0.53      0.52      0.49       146
weighted avg       0.54      0.56      0.52       146



In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model2 = LinearDiscriminantAnalysis() 
model2.fit(features_set, trainY)
testPredict2 = model2.predict(test_features)  
result(testY,testPredict2)

acc: 0.4657534246575342
              precision    recall  f1-score   support

         0.0       0.57      0.31      0.40        84
         1.0       0.42      0.68      0.52        62

    accuracy                           0.47       146
   macro avg       0.49      0.49      0.46       146
weighted avg       0.50      0.47      0.45       146



In [10]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model3 = QuadraticDiscriminantAnalysis() 
model3.fit(features_set, trainY)
testPredict3 = model3.predict(test_features)  
result(testY,testPredict3)

/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


acc: 0.5684931506849316
              precision    recall  f1-score   support

         0.0       0.62      0.67      0.64        84
         1.0       0.49      0.44      0.46        62

    accuracy                           0.57       146
   macro avg       0.55      0.55      0.55       146
weighted avg       0.56      0.57      0.56       146



In [11]:
!pip install xgboost

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [12]:
import xgboost as xgb
model4 = xgb.XGBClassifier()
'''
objective ='reg:logistic', colsample_bytree = 0.5, learning_rate = 0.003,
                max_depth = 20, alpha = 20, n_estimators = 400
'''
model4.fit(features_set, trainY)
testPredict4 = model4.predict(test_features)  
result(testY,testPredict4)

acc: 0.4794520547945205
              precision    recall  f1-score   support

         0.0       0.57      0.38      0.46        84
         1.0       0.42      0.61      0.50        62

    accuracy                           0.48       146
   macro avg       0.50      0.50      0.48       146
weighted avg       0.51      0.48      0.48       146



In [13]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(features_set, trainY)
testPredict5 = model5.predict(test_features)  
result(testY,testPredict5)

acc: 0.4041095890410959
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        84
         1.0       0.41      0.95      0.58        62

    accuracy                           0.40       146
   macro avg       0.21      0.48      0.29       146
weighted avg       0.18      0.40      0.24       146



In [14]:
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=400,criterion="entropy")#n_estimators=400,criterion="entropy"
regr.fit(features_set, trainY)

RandomForestClassifier(criterion='entropy', n_estimators=400)

In [15]:
testPredict6 = regr.predict(test_features)  
result(testY,testPredict6)

acc: 0.4315068493150685
              precision    recall  f1-score   support

         0.0       0.53      0.10      0.16        84
         1.0       0.42      0.89      0.57        62

    accuracy                           0.43       146
   macro avg       0.48      0.49      0.37       146
weighted avg       0.49      0.43      0.34       146



In [18]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 750 kB/s eta 0:00:01
     |████████████████████████████████| 42 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 2.9 MB/s eta 0:00:01


In [19]:
import numpy
import keras
from matplotlib import pyplot
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Conv1D,Flatten
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from keras.models import load_model
%matplotlib inline
from keras import optimizers,initializers
model = Sequential() 

model.add(LSTM(units=96, return_sequences=True, 
               input_shape=(trainX.shape[1],trainX.shape[2])))#
model.add(Dropout(0.3))

model.add(LSTM(units=48, return_sequences=True,activation='relu'))
model.add(Dropout(0.1))
model.add(LSTM(units=48,return_sequences=False,activation='relu'))
model.add(Dropout(0.1))

'''
model.add(Conv1D(kernel_size=20, filters=3,activation='relu'))
model.add(Flatten())
'''

#model.add(Dense(units = 16,activation='relu'))
model.add(Dense(units = 1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',#binary_crossentropy
                  optimizer='adam',#RMSprop optimizers.Adam(lr=0.00001)
                  metrics=['accuracy'])  #mean_squared_error metrics=['accuracy']
history = model.fit(trainX,trainY,epochs=500, batch_size=64, shuffle=True)

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [20]:
testPredict = model.predict_classes(testX)
result(testY,testPredict)

acc: 0.5144230769230769
              precision    recall  f1-score   support

         0.0       0.59      0.42      0.49       115
         1.0       0.47      0.63      0.54        93

    accuracy                           0.51       208
   macro avg       0.53      0.53      0.51       208
weighted avg       0.53      0.51      0.51       208

